In [1]:
from datasets import load_dataset 

# load a custom dataset from local/remote files or folders using the ImageFolder feature

dataset = load_dataset("imagefolder", data_dir="/kaggle/input/mmds-cxr-ii/test", split="train")
dataset

Resolving data files:   0%|          | 0/231 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-ce6c477d5cb10810/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


Dataset({
    features: ['image', 'label'],
    num_rows: 231
})

In [2]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'2-COVID19'

In [3]:
import glob

paths = glob.glob("/kaggle/input/mmds-cxr-ii/test/*/*")
len(paths)

231

In [4]:
import numpy as np

In [5]:
y_true = []

for p in paths:
    f = p.split('/')[5]
    y_true.append(label2id[f])

y_true = np.array(y_true)
y_true.shape

(231,)

In [6]:
from transformers import pipeline

pipe = pipeline("image-classification", "saileshaman/vit-base-patch16-224-in21k-finetuned-cxr")

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [7]:
results = pipe(paths, num_workers=2, batch_size=8, top_k=1)
results[:5]

[[{'score': 0.9827181696891785, 'label': '1-Pneumonia'}],
 [{'score': 0.9900107979774475, 'label': '1-Pneumonia'}],
 [{'score': 0.9497472047805786, 'label': '1-Pneumonia'}],
 [{'score': 0.9882357120513916, 'label': '1-Pneumonia'}],
 [{'score': 0.9901740550994873, 'label': '1-Pneumonia'}]]

In [8]:
y_pred = []

for l in results:
    f = l[0]['label']
    y_pred.append(label2id[f])

y_pred = np.array(y_pred)
y_pred.shape

(231,)

In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Normal", "Pneumonia", "COVID-19"]))

              precision    recall  f1-score   support

      Normal       0.93      0.98      0.96       100
   Pneumonia       0.93      0.95      0.94       100
    COVID-19       1.00      0.77      0.87        31

    accuracy                           0.94       231
   macro avg       0.95      0.90      0.92       231
weighted avg       0.94      0.94      0.94       231

